In [1]:
import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image


Using TensorFlow backend.


In [9]:
DATA_DIR = '/home/chicm/ml/cnnpractices/cervc/data/crop'
TRAIN_DIR = DATA_DIR+'/train'
TEST_DIR = DATA_DIR + '/test'
VAL_DIR = DATA_DIR + '/valid'
RESULT_DIR = DATA_DIR + '/results'

DA_TRAIN_FEAT = RESULT_DIR+'/da_train_feat.dat'
TRAIN_FEAT = RESULT_DIR +'/train_feat.dat'
VAL_FEAT = RESULT_DIR+'/val_feat.dat'
TEST_FEAT = RESULT_DIR+'/test_feat.dat'
TRAIN_LABEL = RESULT_DIR + '/train_label.dat'
VAL_LABEL = RESULT_DIR + '/val_label.dat'
DA_TRAIN_LABEL = RESULT_DIR + '/da_train_label.dat'

batch_size = 32
da = 5
img_size = (224,224)


In [3]:
x=[1,2,3]
x.pop()
print(x)

[1, 2]


In [3]:
img_size = (224,224)
w_file = '/home/chicm/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
w_file2 = '/home/chicm/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5'

def ConvBlock(layers, numlayers, filters):
    for i in range(numlayers):
        layers.append(ZeroPadding2D((1, 1)))
        layers.append(Conv2D(filters, (3, 3), activation='relu'))
    layers.append(MaxPooling2D((2, 2), strides=(2, 2)))


def FCBlock(layers):
    layers.append(Dense(4096, activation='relu'))
    layers.append(Dropout(0.5))


def create_vgg(size):
    layers = []
    #layers.append(Lambda(vgg_preprocess, input_shape=size+(3,), output_shape=size+(3,)))

    ConvBlock(layers, 2, 64)
    ConvBlock(layers, 2, 128)
    ConvBlock(layers, 3, 256)
    ConvBlock(layers, 3, 512)
    ConvBlock(layers, 3, 512)
    
    #layers.append(Flatten())
    #FCBlock(layers)
    #FCBlock(layers)
    #layers.append(Dense(1000, activation='softmax'))
    
    layers[0] = ZeroPadding2D((1,1), input_shape=size+(3,))

    model = Sequential(layers)
    model.load_weights(w_file)
    return model

In [91]:
del model

In [4]:
vgg_model = create_vgg(img_size)

In [5]:
#model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
print(vgg_model.layers[-1].name)

max_pooling2d_5


In [7]:
vgg_model.layers.pop()
print(vgg_model.layers[-1].name)

conv2d_13


In [8]:
model = Sequential(vgg_model.layers)
print(model.layers[-1].name)

conv2d_13


In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

train_datagen = ImageDataGenerator(height_shift_range=0.05,
        shear_range=0.1, channel_shift_range=20, width_shift_range=0.1, horizontal_flip=True, 
        vertical_flip=True, rescale = 1./255)

test_datagen = ImageDataGenerator(rescale=1. / 255)


da_train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical', shuffle = True)

train_generator = test_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical', shuffle = True)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical', shuffle = True)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical', shuffle = False)


Found 7024 images belonging to 3 classes.
Found 7024 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Found 512 images belonging to 1 classes.


In [101]:
print(train_generator.samples)

7024


In [154]:
a = np.array([[1,2]])
b = np.array([[11,21]])
print(np.append(a,b, axis=0))

[[ 1  2]
 [11 21]]


In [12]:
import numpy as np
a=np.array([[0,0]])
b = np.array([[[1,2],[3,4]], [[1,2],[3,4]]])
print(b.shape)
print(b.reshape(4,2))

(2, 2, 2)
[[1 2]
 [3 4]
 [1 2]
 [3 4]]


In [12]:
def get_data(gen, steps):
    x,y = gen.next()
    idx = x.shape[0]
    num = x.shape[0]*steps
    x = np.resize(x, (num,) + x.shape[1:])
    y = np.resize(y, (num,) + y.shape[1:])
    print(x.shape)
    print(y.shape)
    
    for i in range(1, steps):
        print(i)
        x_batch, y_batch = gen.next()
        for j in range(x_batch.shape[0]):
            x[idx] = x_batch[j]
            y[idx] = y_batch[j]
            idx += 1
        #x = np.append(x, x_batch, axis=0)
        #y = np.append(y, y_batch, axis=0)
    return x[:idx], y[:idx]

In [174]:
train_steps = train_generator.samples // batch_size
print(train_steps)

219


In [175]:
train_generator.reset()
x_train, y_train = get_data(train_generator, train_steps)
print(x_train.shape, y_train.shape)

(7008, 224, 224, 3) (7008, 3)


In [172]:
val_steps = validation_generator.samples // batch_size
validation_generator.reset()
x_val, y_val = get_data(validation_generator, val_steps)
print(x_val.shape, y_val.shape)

(576, 224, 224, 3) (576, 3)


In [178]:
del val_feat

In [179]:
train_feat = model.predict(x_train)

In [181]:
save_array(TRAIN_FEAT, train_feat)
save_array(TRAIN_LABEL, y_train)
print(train_feat.shape, y_train.shape)

(7008, 14, 14, 512) (7008, 3)


In [182]:
val_feat = model.predict(x_val)
save_array(VAL_FEAT, val_feat)
save_array(VAL_LABEL, y_val)
print(val_feat.shape, y_val.shape)

(576, 14, 14, 512) (576, 3)


In [13]:
da_train_steps = da_train_generator.samples*da // batch_size
print(da_train_steps)

1097


In [14]:
da_train_generator.reset()
x_train_da, y_train_da = get_data(da_train_generator, da_train_steps)
print(x_train_da.shape, y_train_da.shape)

(35104, 224, 224, 3)
(35104, 3)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269


In [20]:
print(y_train_da[:2])
print(y_train[:2])

[[ 0.  1.  0.]
 [ 0.  1.  0.]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]]


In [17]:
da_train_feat = model.predict(x_train_da)
save_array(DA_TRAIN_FEAT, da_train_feat) 
save_array(DA_TRAIN_LABEL, y_train_da)
print(da_train_feat.shape, y_train_da.shape)

(35040, 14, 14, 512) (35040, 3)


In [18]:
print(da_train_generator.class_indices, train_generator.class_indices)

{'Type_1': 0, 'Type_2': 1, 'Type_3': 2} {'Type_1': 0, 'Type_2': 1, 'Type_3': 2}


In [21]:
y_train=load_array(TRAIN_LABEL)

## train dense model

In [20]:
import bcolz

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

In [25]:
def get_bn_layers(shape):
    return [
        MaxPooling2D(input_shape = shape),
        Flatten(),
        #Dropout(0.25),
        Dense(256, activation='relu'),
        BatchNormalization(),
        #Dropout(0.25),
        Dense(256, activation='relu'),
        BatchNormalization(),
        #Dropout(0.25),
        Dense(3, activation='softmax')
    ]

def get_bn_model(shape):
    bn_model = Sequential(get_bn_layers(shape))
    bn_model.compile(Adam(lr=0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])
    return bn_model

In [10]:
da_train_feat = load_array(DA_TRAIN_FEAT)
y_train_da = load_array(DA_TRAIN_LABEL)
print(da_train_feat.shape, y_train_da.shape)

(35040, 14, 14, 512) (35040, 3)


In [11]:
val_feat = load_array(VAL_FEAT)
y_val = load_array(VAL_LABEL)
print(val_feat.shape, y_val.shape)

(576, 14, 14, 512) (576, 3)


In [26]:
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
def lr_schedule(epoch):
    if epoch <= 10:
        return 0.001
    elif epoch <= 20:
        return 0.01
    elif epoch <= 30:
        return 0.0001
    elif epoch <= 50: 
        return 0.00005
    else:
        return 0.00001

In [27]:
del bn_model

In [28]:
epochs = 300
for i in range(5):
   
    bn_model = get_bn_model(da_train_feat.shape[1:])
    
    w_file = RESULT_DIR+'/w'+str(i)+'.h5'
    
    callbacks=[ModelCheckpoint(w_file, monitor='val_acc', save_best_only=True),
             LearningRateScheduler(lr_schedule)]
    
    bn_model.fit(da_train_feat, y_train_da, batch_size=64, epochs=epochs, verbose=2,
             validation_data=(val_feat, y_val),
             callbacks=callbacks)
    

Train on 35040 samples, validate on 576 samples
Epoch 1/300
15s - loss: 0.9518 - acc: 0.5428 - val_loss: 2.5495 - val_acc: 0.2986
Epoch 2/300
14s - loss: 0.7354 - acc: 0.6719 - val_loss: 0.9450 - val_acc: 0.5573
Epoch 3/300
14s - loss: 0.5686 - acc: 0.7606 - val_loss: 1.0914 - val_acc: 0.5625
Epoch 4/300
14s - loss: 0.4384 - acc: 0.8203 - val_loss: 1.3345 - val_acc: 0.5677
Epoch 5/300
14s - loss: 0.3399 - acc: 0.8636 - val_loss: 1.3189 - val_acc: 0.5764
Epoch 6/300
14s - loss: 0.2749 - acc: 0.8903 - val_loss: 1.8204 - val_acc: 0.5677
Epoch 7/300


KeyboardInterrupt: 